#### Capstone Project- The Battle of Neighborhoods

The objective of this project is to find the best location to start a restaurant in Hyderabad, India. This is done by using Clustering algorithm

# Introduction

Hyderabad, located in the Southern India is fourth largest city in India and is rapidly growing. The city is famous for its varied cuisine and with existing variety of restaurant chains, starting a restaurant is difficult task. 

This project is to find a suitable place in the city to start a restaurant, which of course is dependant on several factors. Some of them are   
    1. Similar restaurants around the place  
    2. The connectivity to the place   
    3. The affordability of the potential customers around the place   
    4. Availability and real estate price of the location, etc.,   
    
This project deals with the fundamental analysis based on location and clustering the restaurants obtained from Foursquare API. The other factors mentioned above are not in the scope of this project though their significance cannot be neglected.


In [2]:
#Importing necessary libraries from python
#install libraries for scraping the data from webpages
!pip install beautifulsoup4 
!pip install lxml
import csv
from bs4 import BeautifulSoup
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import requests
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 


from IPython.display import display_html
import pandas as pd
import numpy as np
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

     |████████████████████████████████| 122kB 6.8MB/s eta 0:00:01
     |████████████████████████████████| 5.5MB 3.4MB/s eta 0:00:01
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.4.5.2 |       hecda079_0         147 KB  conda-forge
    certifi-2020.4.5.2         |   py36h9f0ad1d_0         152 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         395 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              co

In [3]:
pd.set_option('max_colwidth', 800) 

**Collecting Neighborhoods data of Hyderabad**

In [4]:
source = requests.get('https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Hyderabad,_India').text 
soup = BeautifulSoup(source,'lxml') # Web-scraping the data

In [5]:
#Writing the data into csv file
csv_file = open('Hyd.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Neighborhood'])

14

In [6]:
mwcg = soup.find_all(class_ = "mw-category-group")

length = len(mwcg) # Gets the length of number of `mw-category-groups` present

for i in range(1, length):  # Gets all the neighborhoods
    lists = mwcg [i].find_all('a')
    for list in lists:
        nbd = list.get('title') # Gets the title of the neighborhood
        csv_writer.writerow([nbd]) # Writes the name of the neighborhood in the csv file  

In [7]:
#close the csv file
csv_file.close()

In [8]:
df = pd.read_csv('Hyd.csv')
df.head(50)

,Neighborhood
0,Badichowdi
1,Bagh Lingampally
2,Bairamalguda
3,Balkampet
4,Banjara Hills
5,"Bank Street, Hyderabad"
6,"Barkas, Hyderabad"
7,Barkatpura
8,Basheerbagh
9,Bazarghat


The latitude and longitude values are obtained using the geo-coder package, the rows are dropped for which the values are not available.

In [9]:
location = None
latitude = None
longitude = None
index=0
neighbors=[]
for gc in df['Neighborhood']: 
    index=index+1
    try:
        geolocator = Nominatim(user_agent="my-application")
        location = geolocator.geocode('{},Hyderabad,India'.format(gc))
        neighbors.append({'Name':gc,'Latitude': location.latitude,'Longitude': location.longitude})
    except:
        df.drop(index=index,axis=0,inplace=True)

In [10]:
df_hyd=pd.DataFrame(neighbors)
df_hyd.rename(columns={'Name':'Neighborhood'},inplace=True)

Saving the data to a csv file 

In [11]:
df_hyd.to_csv(r'Hyderabad area lat_lng.csv')

In [12]:
df_hyd.head()

,Neighborhood,Latitude,Longitude
0,Badichowdi,17.388375,78.487785
1,Bagh Lingampally,17.397436,78.497971
2,Bairamalguda,17.341062,78.542347
3,Balkampet,17.446923,78.450451
4,Banjara Hills,17.417746,78.439901


**Visualizing Neighborhoods in Hyderabad**

In [13]:
# Hyderabad latitude and longitude using Google search
lat = 17.3850
lng = 78.4867

# Creates map of Kolkata using latitude and longitude values
map_hyd = folium.Map(location=[lat,lng], zoom_start=10,tiles='cartodbpositron')

# Add markers to map
for lat, lng, neighbourhood in zip(df_hyd['Latitude'], df_hyd['Longitude'], df_hyd['Neighborhood']):
    label = ('{}'.format(neighbourhood))
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_hyd)  
    
map_hyd

Client details from foursquare API account. 

In [14]:
CLIENT_ID = 'EMKN3452CXVJIXDRAOEL0J1LTL3RWQIJU2BQGUL3XANREO5M' # your Foursquare ID
CLIENT_SECRET = '5V2YV0QJLSF42CQQFQAQSBUJL55UT5SHYN524WXXTK3BKBAJ' # your Foursquare Secret
VERSION = '20200601' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EMKN3452CXVJIXDRAOEL0J1LTL3RWQIJU2BQGUL3XANREO5M
CLIENT_SECRET:5V2YV0QJLSF42CQQFQAQSBUJL55UT5SHYN524WXXTK3BKBAJ


In [15]:
#Obtain the venues from the API

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
#Limit the values to 100 for computational purpose
LIMIT=100
Hyd_venues = getNearbyVenues(names=df_hyd['Neighborhood'],
                                   latitudes=df_hyd['Latitude'],
                                   longitudes=df_hyd['Longitude']
                                  )


Badichowdi
Bagh Lingampally
Bairamalguda
Balkampet
Banjara Hills
Bank Street, Hyderabad
Barkas, Hyderabad
Barkatpura
Basheerbagh
Bazarghat
Begum Bazaar
Begumpet
Bharat Nagar
BHEL Township, Hyderabad
Borabanda
Bowenpally
Chaderghat
Champapet
Chanchalguda
Chandrayan Gutta
Cherlapally
Chikkadpally
Chilkalguda
Chintal Basti
Dabirpura
Dar-ul-Shifa
Dhoolpet
Dilsukhnagar
Domalguda
Erragadda
Fateh Nagar, Hyderabad
Ferozguda
Film Nagar
Gachibowli
Gaddiannaram
Golnaka
Goshamahal
Gudimalkapur
Gulzar Houz
Habsiguda
Hafeezpet
Hasmathpet
Hastinapuram
Hayathnagar
Himayatnagar, Hyderabad
HITEC City
Hyderguda
Izzat Nagar
Jahanuma
Jamia Osmania
Jeedimetla
Jubilee Hills
Kachiguda
Kakatiya Nagar
Kamala Nagar, Hyderabad
Kapra
Karkhana, Secunderabad
Karmanghat
Karwan
Kavadiguda
Khairtabad
Khajaguda
Khilwat
Kings Colony, Shastripuram
Kompally
Kondapur
Kothapet, Hyderabad
Koti, Hyderabad
Krishna Nagar, Hyderabad
Kukatpally
Kushaiguda
Lab quarters
Lakdi ka pul
Lal Bahadur Nagar
Lal Darwaza
Lallaguda
Langar Hou

In [17]:
print(Hyd_venues.shape) #Shape of the DataFrame
Hyd_venues.head(10)

(984, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Badichowdi,17.388375,78.487785,Inox Maheshwari Paremeshwari,17.390728,78.488352,Multiplex
1,Badichowdi,17.388375,78.487785,Tarakarama Cineplex,17.390854,78.488539,Indie Movie Theater
2,Badichowdi,17.388375,78.487785,Cafe coffee day,17.385343,78.485875,Coffee Shop
3,Badichowdi,17.388375,78.487785,Hotel Panchratan,17.390397,78.491524,Hotel
4,Badichowdi,17.388375,78.487785,Venkatramana Megaplex,17.390465,78.491663,Movie Theater
5,Badichowdi,17.388375,78.487785,KOTI BUS TERMINUS,17.384745,78.485059,Bus Station
6,Bagh Lingampally,17.397436,78.497971,Cafe Coffee Day,17.393138,78.497436,Coffee Shop
7,Bagh Lingampally,17.397436,78.497971,Fitbuzz fitness centre,17.394532,78.500041,Gym
8,Bagh Lingampally,17.397436,78.497971,Gelato de la cream,17.394297,78.496202,Dessert Shop
9,Bagh Lingampally,17.397436,78.497971,kacheguda,17.394590,78.495175,Women's Store


In [18]:
#Writing the venues into a csv file
Hyd_venues.to_csv('Hyd_venues.csv')

In [19]:
Hyd_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"BHEL Township, Hyderabad",7,7,7,7,7,7
Badichowdi,6,6,6,6,6,6
Bagh Lingampally,5,5,5,5,5,5
Bairamalguda,4,4,4,4,4,4
Balkampet,7,7,7,7,7,7
...,...,...,...,...,...,...
Shanker Mutt,4,4,4,4,4,4
Sikh Village,4,4,4,4,4,4
Silpa Avenue Colony,7,7,7,7,7,7


In [20]:
print('There are {} uniques categories.'.format(len(Hyd_venues['Venue Category'].unique())))

There are 154 uniques categories.


In [21]:
Hyd_venues['Venue Category'].unique()

array(['Multiplex', 'Indie Movie Theater', 'Coffee Shop', 'Hotel',
       'Movie Theater', 'Bus Station', 'Gym', 'Dessert Shop',
       "Women's Store", 'Arcade', 'Night Market', 'Garden', 'Bar', 'ATM',
       'Indian Restaurant', 'Train Station', 'Hockey Arena',
       'Light Rail Station', 'Bakery', 'Deli / Bodega', 'Hookah Bar',
       'Café', 'Sandwich Place', 'Clothing Store', 'Trail', 'Shoe Store',
       'South Indian Restaurant', 'Juice Bar', 'Chaat Place',
       'Department Store', 'Snack Place', 'Doner Restaurant',
       'Middle Eastern Restaurant', 'IT Services', 'Bank', 'Platform',
       'Chinese Restaurant', 'Ice Cream Shop', 'Restaurant',
       'Fast Food Restaurant', 'Lake', 'Smoke Shop', 'Breakfast Spot',
       'Gaming Cafe', 'Hotel Bar', 'Food Court', 'Pharmacy',
       'Gym / Fitness Center', 'Business Service', 'Jewelry Store',
       'Gift Shop', 'Sporting Goods Shop', 'Harbor / Marina',
       'Hyderabadi Restaurant', 'Vegetarian / Vegan Restaurant', 'Food',
 

Eat-outs are classified under many names in India, which generally serve everything.Hence I added all names under for the analysis.

In [22]:
a=Hyd_venues[Hyd_venues['Venue Category'].str.contains('Restaurant')]
b=Hyd_venues[Hyd_venues['Venue Category'].str.contains('Café')]
c=Hyd_venues[Hyd_venues['Venue Category'].str.contains('Food')]
d=Hyd_venues[Hyd_venues['Venue Category'].str.contains('Sandwich')]
a=pd.concat([a,b,c,d])


In [23]:
# one hot encoding
hyd_onehot=pd.get_dummies(a[['Venue Category']],prefix="",prefix_sep="")
# add neighborhood column back to dataframe
hyd_neigh= a['Neighborhood'] 
hyd_onehot.insert(0,'Neighborhood',hyd_neigh)

In [24]:
hyd_onehot.reset_index(drop=True,inplace=True)

In [25]:
hyd_grouped = hyd_onehot.groupby('Neighborhood').mean().reset_index()
hyd_grouped

,Neighborhood,Afghan Restaurant,American Restaurant,Asian Restaurant,Bengali Restaurant,Café,Chinese Restaurant,Comfort Food Restaurant,Doner Restaurant,Dumpling Restaurant,...,Multicuisine Indian Restaurant,New American Restaurant,North Indian Restaurant,Punjabi Restaurant,Rajasthani Restaurant,Restaurant,Sandwich Place,South Indian Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant
0,"BHEL Township, Hyderabad",0.0,0.0,0.000000,0.0,0.000000,1.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.00,0.0,0.0
1,Balkampet,0.0,0.0,0.000000,0.0,0.000000,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.00,0.0,0.0
2,Banjara Hills,0.0,0.0,0.000000,0.0,0.400000,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.200,0.00,0.0,0.0
3,"Bank Street, Hyderabad",0.0,0.0,0.000000,0.0,0.000000,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.25,0.0,0.0
4,"Barkas, Hyderabad",0.0,0.0,0.000000,0.0,0.000000,0.00,0.0,0.5,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,Shahran Market,0.0,0.0,0.333333,0.0,0.333333,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.00,0.0,0.0
100,Sikh Village,0.0,0.0,0.000000,0.0,0.500000,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.00,0.0,0.0
101,Silpa Avenue Colony,0.0,0.0,0.200000,0.0,0.000000,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.00,0.0,0.0
102,"Sindhi Colony, Secunderabad",0.0,0.0,0.000000,0.0,0.250000,0.25,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.125,0.00,0.0,0.0


## Clustering the Restaurants

In [26]:
# set number of clusters
kclusters = 3

hyd_clustering = hyd_grouped.drop(["Neighborhood"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(hyd_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 2, 1, 2, 1, 0, 1, 1, 1, 1], dtype=int32)

In [27]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
hyd_merged = hyd_grouped.copy()

# add clustering labels
hyd_merged["Cluster Labels"] = kmeans.labels_
hyd_merged

,Neighborhood,Afghan Restaurant,American Restaurant,Asian Restaurant,Bengali Restaurant,Café,Chinese Restaurant,Comfort Food Restaurant,Doner Restaurant,Dumpling Restaurant,...,New American Restaurant,North Indian Restaurant,Punjabi Restaurant,Rajasthani Restaurant,Restaurant,Sandwich Place,South Indian Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Cluster Labels
0,"BHEL Township, Hyderabad",0.0,0.0,0.000000,0.0,0.000000,1.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000,0.00,0.0,0.0,1
1,Balkampet,0.0,0.0,0.000000,0.0,0.000000,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000,0.00,0.0,0.0,2
2,Banjara Hills,0.0,0.0,0.000000,0.0,0.400000,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.200,0.00,0.0,0.0,1
3,"Bank Street, Hyderabad",0.0,0.0,0.000000,0.0,0.000000,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000,0.25,0.0,0.0,2
4,"Barkas, Hyderabad",0.0,0.0,0.000000,0.0,0.000000,0.00,0.0,0.5,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000,0.00,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,Shahran Market,0.0,0.0,0.333333,0.0,0.333333,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000,0.00,0.0,0.0,1
100,Sikh Village,0.0,0.0,0.000000,0.0,0.500000,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000,0.00,0.0,0.0,0
101,Silpa Avenue Colony,0.0,0.0,0.200000,0.0,0.000000,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000,0.00,0.0,0.0,1
102,"Sindhi Colony, Secunderabad",0.0,0.0,0.000000,0.0,0.250000,0.25,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.125,0.00,0.0,0.0,1


In [28]:
# merge hyd_grouped with hyd_data to add latitude/longitude for each neighborhood
hyd_merged = hyd_merged.join(df_hyd.set_index("Neighborhood"), on="Neighborhood")

print(hyd_merged.shape)
hyd_merged.head() # check the last columns!

(104, 36)


,Neighborhood,Afghan Restaurant,American Restaurant,Asian Restaurant,Bengali Restaurant,Café,Chinese Restaurant,Comfort Food Restaurant,Doner Restaurant,Dumpling Restaurant,...,Punjabi Restaurant,Rajasthani Restaurant,Restaurant,Sandwich Place,South Indian Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Cluster Labels,Latitude,Longitude
0,"BHEL Township, Hyderabad",0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,1,17.486255,78.313355
1,Balkampet,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,2,17.446923,78.450451
2,Banjara Hills,0.0,0.0,0.0,0.0,0.4,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.2,0.00,0.0,0.0,1,17.417746,78.439901
3,"Bank Street, Hyderabad",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.25,0.0,0.0,2,17.385717,78.480157
4,"Barkas, Hyderabad",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,1,17.315704,78.476250


**Visualizing the Clusters**

In [29]:
map_clusters = folium.Map(location=[lat, lng], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(hyd_merged['Latitude'], hyd_merged['Longitude'], hyd_merged['Neighborhood'], hyd_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [30]:
print(hyd_merged.shape)
hyd_merged.sort_values(["Cluster Labels"], inplace=True)
hyd_merged

(104, 36)


,Neighborhood,Afghan Restaurant,American Restaurant,Asian Restaurant,Bengali Restaurant,Café,Chinese Restaurant,Comfort Food Restaurant,Doner Restaurant,Dumpling Restaurant,...,Punjabi Restaurant,Rajasthani Restaurant,Restaurant,Sandwich Place,South Indian Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Cluster Labels,Latitude,Longitude
49,Lal Darwaza,0.0,0.0,0.0,0.0,0.50,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,17.346386,78.477451
32,Jamia Osmania,0.0,0.0,0.0,0.0,1.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,17.410264,78.518037
67,Moula-Ali,0.0,0.0,0.0,0.0,0.50,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,17.466816,78.559216
35,Kachiguda,0.0,0.0,0.0,0.0,1.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,17.390040,78.494621
64,Moghalpura,0.0,0.0,0.0,0.0,0.50,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,17.357528,78.477084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,Masab Tank,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,17.402962,78.450754
42,"Kings Colony, Shastripuram",0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,17.333614,78.447376
43,Kompally,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,17.544703,78.491842
14,Chandrayan Gutta,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,17.323929,78.479866


# Examining the Clusters

In [31]:
hyd_merged.loc[hyd_merged['Cluster Labels'] == 0]

,Neighborhood,Afghan Restaurant,American Restaurant,Asian Restaurant,Bengali Restaurant,Café,Chinese Restaurant,Comfort Food Restaurant,Doner Restaurant,Dumpling Restaurant,...,Punjabi Restaurant,Rajasthani Restaurant,Restaurant,Sandwich Place,South Indian Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Cluster Labels,Latitude,Longitude
49,Lal Darwaza,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,17.346386,78.477451
32,Jamia Osmania,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,17.410264,78.518037
67,Moula-Ali,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,17.466816,78.559216
35,Kachiguda,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,17.390040,78.494621
64,Moghalpura,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,17.357528,78.477084
38,Karwan,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,17.380082,78.437475
28,Hasmathpet,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,17.476170,78.482477
12,Chaderghat,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,17.382033,78.492534
11,Bowenpally,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,17.475537,78.479228
59,Manovikas Nagar,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,17.468330,78.485153


In [32]:
hyd_merged.loc[hyd_merged['Cluster Labels'] == 1]

,Neighborhood,Afghan Restaurant,American Restaurant,Asian Restaurant,Bengali Restaurant,Café,Chinese Restaurant,Comfort Food Restaurant,Doner Restaurant,Dumpling Restaurant,...,Punjabi Restaurant,Rajasthani Restaurant,Restaurant,Sandwich Place,South Indian Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Cluster Labels,Latitude,Longitude
58,Manikonda,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.666667,0.000000,0.0,0.0,0.0,1,17.400002,78.386190
70,Nalgonda 'X' Roads,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,1,17.375093,78.498412
69,Mylargadda,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,1,17.422291,78.516970
68,Musheerabad,0.000000,0.0,0.000000,0.0,0.333333,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,1,17.419142,78.498573
53,Madhapur,0.071429,0.0,0.142857,0.0,0.071429,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.071429,0.0,0.0,0.0,1,17.440858,78.391629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37,"Karkhana, Secunderabad",0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,1.000000,0.000000,0.0,0.0,0.0,1,17.473214,78.511502
36,Kakatiya Nagar,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.333333,0.333333,0.0,0.0,0.0,1,17.414686,78.539765
6,Basheerbagh,0.000000,0.0,0.000000,0.0,0.100000,0.3,0.0,0.0,0.0,...,0.0,0.0,0.200000,0.000000,0.0,0.0,0.0,1,17.405002,78.476910
34,Jubilee Hills,0.000000,0.0,0.000000,0.0,0.125000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,1,17.430836,78.410288


In [33]:
hyd_merged.loc[hyd_merged['Cluster Labels'] == 2]

,Neighborhood,Afghan Restaurant,American Restaurant,Asian Restaurant,Bengali Restaurant,Café,Chinese Restaurant,Comfort Food Restaurant,Doner Restaurant,Dumpling Restaurant,...,Punjabi Restaurant,Rajasthani Restaurant,Restaurant,Sandwich Place,South Indian Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Cluster Labels,Latitude,Longitude
95,Saifabad,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.166667,0.0,0.166667,2,17.404788,78.465837
92,Rastrapathi Road,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,2,17.428560,78.487940
1,Balkampet,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,2,17.446923,78.450451
3,"Bank Street, Hyderabad",0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.250000,0.0,0.000000,2,17.385717,78.480157
77,Neredmet,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,2,17.470446,78.536005
88,RTC X Roads,0.0,0.0,0.333333,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,2,17.407032,78.496802
18,Ferozguda,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,2,17.464618,78.461633
81,Padmarao Nagar,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,2,17.425601,78.508515
21,Golnaka,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,2,17.390106,78.508360
79,Old Neredmet,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,2,17.470446,78.536005


# **Discussion**  

The places in the cluster 2 have many restaurants and are similar to one another in terms of availability of restaurants and eat-outs in Hyderabad. People are overloaded with various choice of restaurants in their neighbourhood. Hence, restaurateurs need to be extremely innovative in service and creating environments, which often is capital intensive. 

The majority of the areas in cluster 3 are located in the older part of the city which habitats people from below middle class section of society. This can be a good choice for eat-outs offering services at less cost as preferred by the general public. 

The areas in cluster 1 accommodates middle and above middle class people, in the heart of the city which have shown considerable growth in the past ten to twelve years. This can be a good spot for restaurants to open.

# **Conclusion**   
Cluster 1 is a very good choice to attract customers, as there are not many restaurants around. People in these areas need not travel far or depend on online deliveries for good food anymore. In addition, residents who were not keen to travel for food before can now visit the restaurants in this area. 

With minimum capital investment, as these areas are not very expensive, owners can serve the customers. Hence this is a win-win situation for customers as well restaurant owners.
